# <span style="color:#0b486b">FIT3162 MCS13 Code</span>


---

## <span style="color:#0b486b">Imports</span>

Library imports required for this file. 
- `os` - OS related functions (getting/setting current directory or filepath(s))
- `zipfile` - to unzip files
- `PIL.Image` -  Image related functions
- `sklearn.model_selection.train_test_split` - A function to split up datasets
- `tensorflow` - General Deep Learning purposes
  - `keras`
    - `layers, models, activations` - for Deep Learning Model creation
- `matplotlib.pyplot` - for plotting graphs (if needed)
- `numpy` - for array types (needed for tensorflow functions)
- `BaseModel` - custom CNN Model class
- `constants` - Constant values used throughout the file

In [1]:
""" Code Header
This file contains code thats trains and tests Models for Image Classification.
Various Model architectures are used and tried. 

In order to make use of the dataset files, ensure that the download.py file under the RealWorldOccludedFaces-main directory is run already.

Note: 
This jupyter file was obtained and altered from the Google Colab file from the MCS13 private Google Drive
The file was created on 08/03/2024 and contained code cells for: 
- Image loading
- Image resizing
- Creation of some simple models (obtained and altered from GeminiAI and Microsoft Copilot)
- Model training

@author MCS13
@version 1.0.0
@since 29/03/2024
"""

# ============================================================================================================= #

# Imports 
import os
import zipfile
from PIL import Image
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, activations

import matplotlib.pyplot as plt
import numpy as np

from typing import List, Tuple
from numpy.typing import NDArray

from BaseModel import BaseModel
from constants import *

plt.style.use('ggplot')
%matplotlib inline


---

## <span style="color:#0b486b">Loading Datasets</span>

***[RUN ONCE]*** Unzip and extract all files.

In [2]:
def unzip_file(filepath: str):
    """Unzips a zip file
    This code unzips a file and saves the content in the current directory.
    This means the content will be under the same directory as the original zip file.

    @since 1.0.0

    @prerequisite: The input filepath must end with ".zip"
    @raise ValueError: When the input filepath does not end with ".zip"

    @param filepath(str): The file path to unzip. The file path must lead to a zip file
    """
    # Pre-requisite check
    if filepath[-4:] != ".zip":
        raise ValueError(
            f"The input filepath must end with '.zip'. Expected filepath = '<FileName>.zip', got filepath = '{filepath}'."
        )

    # Unzip the file
    zip_obj = zipfile.ZipFile(file=filepath, mode="r")
    zip_obj.extractall("./")
    zip_obj.close()


"""RUN ONCE: Unzipping the dataset zip file """
# dataset_zip_filepath = f"{DATASET_NAME_MAIN}.zip"
# unzip_file(dataset_zip_filepath)

'RUN ONCE: Unzipping the dataset zip file '

Here, we start with the neutral faces dataset.

In [3]:
print(
    f'A few samples of people in the dataset: {os.listdir("RealWorldOccludedFaces-main/images/neutral")[:5]}'
)
print(
    f'Number of unique ids: {len(os.listdir("RealWorldOccludedFaces-main/images/neutral"))}'
)

A few samples of people in the dataset: ['adrien_brody', 'alain_delon', 'alexander_zverev', 'al_pacino', 'amber_heard']
Number of unique ids: 182


***[RUN ONCE]*** Resizes all images to a fixed shape of (64, 64)

In [5]:
# Main file paths
main_path = f"{DATASET_NAME_MAIN}/{NEUTRAL_DIR}"
main_dirs = os.listdir(main_path)
num_classes = len(main_dirs)

def resize_neutral_images(resized_shape: Tuple[int, int]):
    """Resizes all neutral images
    The function takes all images in RealWorldOccludedFaces-main/images/neutral
      and resizes them to 224x224 (determined by RESIZED_SHAPE constant)

    The resized shapes are then saved in RealWorldOccludedFaces-resized/images/neutral

    @param resized_shape(Tuple[int, int]): The shape to resize the shape to.
    """
    # Creates the directory paths
    for path in [os.path.join(resized_path, f) for f in os.listdir(main_path)]:
        os.makedirs(f"{path}")
    
    resized_path = f"{DATASET_NAME_RESIZED}_{resized_shape[0]}/{NEUTRAL_DIR}"

    # File paths of the neutral images
    paths = [
        os.path.join(resized_path, f)
        for f in main_dirs
    ]

    # Resizing and saving all images
    for name in os.listdir(main_path):
        for img_name in os.listdir(f"{main_path}/{name}"):
            Image.open(os.path.join(f"{main_path}/{name}/", img_name)).resize(
                RESIZED_SHAPE
            ).save(os.path.join(f"{resized_path}/{name}/", img_name))


# Creating the directory paths before resizing the images
# Needed before saving files
"""RUN ONCE: Resizing image dataset"""
# for path in [os.path.join(resized_path, f) for f in os.listdir(main_path)]:
#     os.makedirs(f"{path}")

# resize_neutral_images()

Function to load the images into numpy arrays for model input.

In [6]:
def get_neutral_image_data() -> (
    Tuple[NDArray[List[List[int]]], NDArray[NDArray[List[List[int]]]]]
):
    """Generates the dataset values (as ndarrays)
    Produces the dataset values (x-values) and
        the one-hot vector encoding of their labels (y-values)

    The values are obtained from the neutral images

    @rtype: Tuple[ndarray[List[List[int]]], ndarray[ndarray[List[List[uint8]]]]]
    @return: x-values, y-values
    """
    # Return the dataset values
    return np.array(
        [
            np.array(Image.open(img), "uint8")
            for path in paths
            for img in [os.path.join(path, f) for f in os.listdir(path)]
        ]
    ), np.array(
        [
            [int(bit == unique_id) for bit in range(len(resized_dirs))]
            for unique_id, path in enumerate(paths)
            for _ in [os.path.join(path, f) for f in os.listdir(path)]
        ]
    )

In [7]:
# Takes a while...
faces, ids = get_neutral_image_data()

In [8]:
print(f"Number of face images: {len(faces)}")  # size of neutral faces dataset
print(f"Number of ids (must match above): {len(ids)}")  # size of the array of face ids

Number of face images: 5927
Number of ids (must match above): 5927


Splitting the dataset into training, validation and testing.

In [9]:
# Splitting the dataset into training and testing datasets
x_train, x_test, y_train, y_test = train_test_split(
    faces, ids, test_size=0.2, random_state=42
)

# Splitting the training dataset into training and validation datasets
x_train, x_val, y_train, y_val = train_test_split(
    x_train, y_train, test_size=0.1, random_state=42
)


--- 

## <span style="color:#0b486b">Modelling time!</span>

In [10]:
model = BaseModel(
    name="Basic Model",
    input_width=WIDTH,
    input_height=HEIGHT,
    depth=DEPTH,
    num_classes=num_classes,
    activation_func=RELU,
    optimiser=ADAM_OPT,
)

model.build_cnn()
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 224, 224, 32)      896       
                                                                 
 conv2d_1 (Conv2D)           (None, 224, 224, 32)      9248      
                                                                 
 average_pooling2d (AverageP  (None, 112, 112, 32)     0         
 ooling2D)                                                       
                                                                 
 conv2d_2 (Conv2D)           (None, 112, 112, 64)      18496     
                                                                 
 conv2d_3 (Conv2D)           (None, 112, 112, 64)      36928     
                                                                 
 average_pooling2d_1 (Averag  (None, 56, 56, 64)       0         
 ePooling2D)                                            

In [11]:
model.fit(x_train=x_train, y_train=y_train, x_val=x_val, y_val=y_val)

Epoch 1/20
 71/134 [==============>...............] - ETA: 2:59 - loss: 25.8086 - accuracy: 0.0286

KeyboardInterrupt: 

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, activations

activation_func = "relu"

input_shape = (64, 64, 1)

model = Sequential()
model.add(layers.Conv2D(32, (3,3), padding='same', activation=activation_func, input_shape=input_shape))
model.add(layers.Conv2D(32, (3,3), padding='same', activation=activation_func))
model.add(layers.AveragePooling2D(pool_size=(2, 2), padding='same'))
model.add(layers.Conv2D(64, (3,3), padding='same', activation=activation_func))
model.add(layers.Conv2D(64, (3,3), padding='same', activation=activation_func))
model.add(layers.AveragePooling2D(pool_size=(2, 2), padding='same'))
model.add(layers.Flatten())
model.add(layers.Dense(num_classes, activation='softmax'))
model.compile(optimizer="adam", loss='categorical_crossentropy', metrics=['accuracy'])

Train the model

In [ ]:
# Compile the model
# model.compile(loss=keras.losses.categorical_crossentropy,
#               optimizer=keras.optimizers.Adadelta(),
#               metrics=['accuracy'])

model.fit(np.array(X_train), y_train, batch_size=32, epochs=20, verbose=1, validation_data=(np.array(X_valid), y_valid))

In [ ]:
evaluation_results = model.evaluate(np.array(X_test), y_test)
metrics = ['loss', 'accuracy']
# Print the evaluation results
for metric, result in zip(metrics, evaluation_results):
    print(f'{metric}: {result}')